# LangChain Middleware

Ref: https://docs.langchain.com/oss/python/langchain/middleware/overview

Middleware allows you to control and customize agent execution at every step.


## Setup

Configure `.env` before running. See `.env.sample`.


In [1]:
import rich
from dotenv import load_dotenv

load_dotenv()

True

## Custom Middleware: Logging All Hooks

Create a middleware that logs every hook to understand the execution flow.


In [2]:
from collections.abc import Callable
from datetime import datetime
from typing import Any

from langchain.agents import create_agent
from langchain.agents.middleware.types import (
    AgentMiddleware,
    AgentState,
    ToolCallRequest,
)
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langgraph.graph.state import CompiledStateGraph
from langgraph.runtime import Runtime
from langgraph.types import Command


class LoggingMiddleware(AgentMiddleware[AgentState[Any], None]):
    """Middleware that logs all hooks."""

    def before_agent(self, state: AgentState[Any], runtime: Runtime[None]) -> dict[str, Any] | None:
        rich.print("[bold blue]>>> before_agent[/bold blue]")
        rich.print("state =", state)
        rich.print("runtime =", runtime)
        return None

    def before_model(self, state: AgentState[Any], runtime: Runtime[None]) -> dict[str, Any] | None:
        rich.print("[bold green]>>> before_model[/bold green]")
        rich.print("state =", state)
        rich.print("runtime =", runtime)
        return None

    def after_model(self, state: AgentState[Any], runtime: Runtime[None]) -> dict[str, Any] | None:
        rich.print("[bold green]<<< after_model[/bold green]")
        rich.print("state =", state)
        rich.print("runtime =", runtime)
        return None

    def wrap_tool_call(
        self,
        request: ToolCallRequest,
        handler: Callable[[ToolCallRequest], ToolMessage | Command[Any]],
    ) -> ToolMessage | Command[Any]:
        rich.print("[bold yellow]>>> wrap_tool_call (before)[/bold yellow]")
        rich.print("request =", request)
        result = handler(request)
        rich.print("[bold yellow]<<< wrap_tool_call (after)[/bold yellow]")
        rich.print("result =", result)
        return result

    def after_agent(self, state: AgentState[Any], runtime: Runtime[None]) -> dict[str, Any] | None:
        rich.print("[bold blue]<<< after_agent[/bold blue]")
        rich.print("state =", state)
        rich.print("runtime =", runtime)
        return None


@tool
def get_current_time() -> str:
    """Get the current time."""
    return datetime.now().isoformat()


model = ChatAnthropic(model="claude-sonnet-4-5-20250929")
agent: CompiledStateGraph[Any] = create_agent(
    model=model,
    tools=[get_current_time],
    system_prompt="You are a helpful assistant.",
    middleware=[LoggingMiddleware()],
)

In [3]:
response = agent.invoke({"messages": [{"role": "user", "content": "What time is it?"}]})
rich.print("response =", response)

>>> before_agent

state =
{
    'messages': [
        HumanMessage(
            content='What time is it?',
            additional_kwargs={},
            response_metadata={},
            id='da5820f3-014b-4f6b-8520-55f1a14cd89d'
        )
    ]
}

runtime =
Runtime(
    context=None,
    store=None,
    stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x10f8405e0>,
    previous=None
)

>>> before_model

state =
{
    'messages': [
        HumanMessage(
            content='What time is it?',
            additional_kwargs={},
            response_metadata={},
            id='da5820f3-014b-4f6b-8520-55f1a14cd89d'
        )
    ]
}

...

runtime =
Runtime(
    context=None,
    store=None,
    stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x10f8405e0>,
    previous=None
)

<<< after_agent

state =
{
    'messages': [
        HumanMessage(
            content='What time is it?',
            additional_kwargs={},
            response_metadata={},
            id='da5820f3-014b-4f6b-8520-55f1a14cd89d'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_01PbcfXo9S1qwA7o7eNs2ZDB',
                    'input': {},
                    'name': 'get_current_time',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_012HPcqE4tyUxDUMbJGtxe11',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 555,
                    'output_tokens': 38,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--cce12dd8-c135-44a7-ae76-d35cd89217d8-0',
            tool_calls=[
                {
                    'name': 'get_current_time',
                    'args': {},
                    'id': 'toolu_01PbcfXo9S1qwA7o7eNs2ZDB',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 555,
                'output_tokens': 38,
                'total_tokens': 593,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        ),
        ToolMessage(
            content='2025-12-05T07:41:59.107322',
            name='get_current_time',
            id='bd77b651-d924-4dbc-a05e-7eb906b3f82b',
            tool_call_id='toolu_01PbcfXo9S1qwA7o7eNs2ZDB'
        ),
        AIMessage(
            content='The current time is **7:41:59 AM** on December 5th, 2025.',
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01DCMHBDuKjqHkb6d2Gg8Uw3',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'end_turn',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 621,
                    'output_tokens': 26,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--8728465d-2f06-4497-9e69-2aea62f2ab37-0',
            usage_metadata={
                'input_tokens': 621,
                'output_tokens': 26,
                'total_tokens': 647,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        )
    ]
}

runtime =
Runtime(
    context=None,
    store=None,
    stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x10f8405e0>,
    previous=None
)

response =
{
    'messages': [
        HumanMessage(
            content='What time is it?',
            additional_kwargs={},
            response_metadata={},
            id='da5820f3-014b-4f6b-8520-55f1a14cd89d'
        ),
        AIMessage(
            content=[
                {
                    'id': 'toolu_01PbcfXo9S1qwA7o7eNs2ZDB',
                    'input': {},
                    'name': 'get_current_time',
                    'type': 'tool_use'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'id': 'msg_012HPcqE4tyUxDUMbJGtxe11',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'tool_use',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 555,
                    'output_tokens': 38,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--cce12dd8-c135-44a7-ae76-d35cd89217d8-0',
            tool_calls=[
                {
                    'name': 'get_current_time',
                    'args': {},
                    'id': 'toolu_01PbcfXo9S1qwA7o7eNs2ZDB',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 555,
                'output_tokens': 38,
                'total_tokens': 593,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        ),
        ToolMessage(
            content='2025-12-05T07:41:59.107322',
            name='get_current_time',
            id='bd77b651-d924-4dbc-a05e-7eb906b3f82b',
            tool_call_id='toolu_01PbcfXo9S1qwA7o7eNs2ZDB'
        ),
        AIMessage(
            content='The current time is **7:41:59 AM** on December 5th, 2025.',
            additional_kwargs={},
            response_metadata={
                'id': 'msg_01DCMHBDuKjqHkb6d2Gg8Uw3',
                'model': 'claude-sonnet-4-5-20250929',
                'stop_reason': 'end_turn',
                'stop_sequence': None,
                'usage': {
                    'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0},
                    'cache_creation_input_tokens': 0,
                    'cache_read_input_tokens': 0,
                    'input_tokens': 621,
                    'output_tokens': 26,
                    'server_tool_use': None,
                    'service_tier': 'standard'
                },
                'model_name': 'claude-sonnet-4-5-20250929',
                'model_provider': 'anthropic'
            },
            id='lc_run--8728465d-2f06-4497-9e69-2aea62f2ab37-0',
            usage_metadata={
                'input_tokens': 621,
                'output_tokens': 26,
                'total_tokens': 647,
                'input_token_details': {
                    'cache_read': 0,
                    'cache_creation': 0,
                    'ephemeral_5m_input_tokens': 0,
                    'ephemeral_1h_input_tokens': 0
                }
            }
        )
    ]
}